### Basic Imports

In [20]:
import matplotlib.pyplot as plt
import torch
import gymnasium as gym
import numpy as np

%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
# List of environments
list(gym.envs.registry.keys())

['CartPole-v0',
 'CartPole-v1',
 'MountainCar-v0',
 'MountainCarContinuous-v0',
 'Pendulum-v1',
 'Acrobot-v1',
 'phys2d/CartPole-v0',
 'phys2d/CartPole-v1',
 'phys2d/Pendulum-v0',
 'LunarLander-v2',
 'LunarLanderContinuous-v2',
 'BipedalWalker-v3',
 'BipedalWalkerHardcore-v3',
 'CarRacing-v2',
 'Blackjack-v1',
 'FrozenLake-v1',
 'FrozenLake8x8-v1',
 'CliffWalking-v0',
 'Taxi-v3',
 'tabular/Blackjack-v0',
 'tabular/CliffWalking-v0',
 'Reacher-v2',
 'Reacher-v4',
 'Pusher-v2',
 'Pusher-v4',
 'InvertedPendulum-v2',
 'InvertedPendulum-v4',
 'InvertedDoublePendulum-v2',
 'InvertedDoublePendulum-v4',
 'HalfCheetah-v2',
 'HalfCheetah-v3',
 'HalfCheetah-v4',
 'Hopper-v2',
 'Hopper-v3',
 'Hopper-v4',
 'Swimmer-v2',
 'Swimmer-v3',
 'Swimmer-v4',
 'Walker2d-v2',
 'Walker2d-v3',
 'Walker2d-v4',
 'Ant-v2',
 'Ant-v3',
 'Ant-v4',
 'Humanoid-v2',
 'Humanoid-v3',
 'Humanoid-v4',
 'HumanoidStandup-v2',
 'HumanoidStandup-v4',
 'GymV21Environment-v0',
 'GymV26Environment-v0']

In [3]:
env = gym.make("CartPole-v0", render_mode="human")
observation, info = env.reset(seed=42)

/Users/nipun/miniconda3/lib/python3.9/site-packages/gymnasium/envs/registration.py:513: DeprecationWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  logger.deprecation(


In [4]:
env.action_space

Discrete(2)

In [5]:
env.action_space.sample()

0

In [6]:
env.action_space.sample()

0

In [7]:
print(env.observation_space)

Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


In [8]:
for _ in range(100):
   action = env.action_space.sample()  # this is where you would insert your policy
   observation, reward, terminated, truncated, info = env.step(action)

   if terminated or truncated:
      observation, info = env.reset()



In [12]:
# Now, let us take only the action 1
observation, info = env.reset(seed=42)
for i in range(100):
   action = 0  # this is where you would insert your policy
   observation, reward, terminated, truncated, info = env.step(action)

   if terminated or truncated:
        print("Episode finished after {} timesteps".format(i+1))
        observation, info = env.reset()

Episode finished after 8 timesteps
Episode finished after 17 timesteps
Episode finished after 27 timesteps
Episode finished after 37 timesteps
Episode finished after 46 timesteps
Episode finished after 54 timesteps
Episode finished after 64 timesteps
Episode finished after 73 timesteps
Episode finished after 83 timesteps
Episode finished after 93 timesteps


In [26]:
# Create a q table
num_states = env.observation_space.shape[0]
num_actions = env.action_space.n


In [49]:
# Define the number of bins for each dimension
num_bins = [16, 16, 16, 16]  # Adjust these values based on your preference

# Discretize the continuous state space
def discretize_state(state, num_bins):
    state_discrete = []
    for i in range(len(state)):
        bins = np.linspace(env.observation_space.low[i], env.observation_space.high[i], num_bins[i] + 1)[1:-1]
        state_discrete.append(np.digitize(state[i], bins))
    return tuple(state_discrete)


# Initialize Q-table with zeros
q_table = torch.zeros(num_bins + [env.action_space.n])

def update_q_table(q_table, state, action, reward, next_state, learning_rate, discount_factor):
    state = tuple(state)
    next_state = tuple(next_state)
    q_table[state][action] = (1 - learning_rate) * q_table[state][action] + learning_rate * (reward + discount_factor * torch.max(q_table[next_state]))

    return q_table

In [50]:
# Learn the q table

# Hyperparameters
learning_rate = 0.1
discount_factor = 0.9


In [51]:
# Learn q-table without epsilon greedy approach and print rewards
num_episodes = 10
render_mode = False

    

In [52]:
# Print all possible states
print("All possible states:")
print(env.observation_space)

All possible states:
Box([-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38], [4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38], (4,), float32)


In [53]:
# Print some sample states
for _ in range(10):
    state = env.observation_space.sample()
    print("Sample State:", state)

Sample State: [-8.2706535e-01 -2.8081686e+38 -3.0893192e-01 -9.7170763e+37]
Sample State: [ 1.9736170e+00 -2.1225781e+38 -1.8008137e-01  4.6366904e+36]
Sample State: [-1.7399893e+00  5.8333194e+37 -5.9940118e-02 -2.7254928e+38]
Sample State: [-3.8519826e+00  2.2531339e+38  6.3289993e-02  9.1026116e+37]
Sample State: [ 2.5397954e+00 -1.2115095e+38  2.4115197e-01 -1.3963663e+38]
Sample State: [3.6893094e+00 3.1494561e+37 2.1339690e-02 2.6938287e+38]
Sample State: [ 3.2560833e+00  1.3971532e+38  1.6714491e-02 -2.7917394e+38]
Sample State: [5.7502143e-02 1.3929552e+38 3.0328104e-01 3.1058507e+38]
Sample State: [ 7.9759490e-01 -2.2935894e+38 -4.7575418e-02 -9.9444867e+37]
Sample State: [-4.6630378e+00  4.8043669e+37 -1.4580821e-01  2.4613882e+38]


In [54]:
# Training loop
for episode in range(num_episodes):
    state = discretize_state(env.reset(), num_bins)

    while True:
        # Choose action using the current Q-table
        action = torch.argmax(q_table[state]).item()

        # Take the chosen action and observe the next state and reward
        next_state, reward, done, _ = env.step(action)
        next_state = discretize_state(next_state, num_bins)

        # Update the Q-table using the Q-learning update rule
        q_table = update_q_table(q_table, state, action, reward, next_state, learning_rate, discount_factor)

        state = next_state

        if done:
            break

# Print the learned Q-table
print("Learned Q-table:")
print(q_table)

TypeError: '<' not supported between instances of 'dict' and 'dict'

In [55]:
discretize_state(env.reset(), num_bins)

TypeError: '<' not supported between instances of 'dict' and 'dict'

In [36]:
q_table

tensor([[ 0.0982, -0.0016],
        [ 0.0100,  0.0040],
        [-0.0006,  0.0108],
        [ 0.0028,  0.0059]])

In [37]:
q_table[state]

IndexError: tensors used as indices must be long, int, byte or bool tensors